In [11]:
import pymongo
import pandas as pd
from pymongo import MongoClient

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
client = MongoClient()

#point the client at mongo URI
client = MongoClient('mongodb://localhost:27017')
#select database
db = client['Yelp_Database']
print(db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Yelp_Database')


In [13]:
import os
import re
import string

import numpy as np
import pandas as pd

from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer

In [14]:
df_business = db['recommender_review_joined']
print(df_business)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Yelp_Database'), 'recommender_review_joined')


In [15]:
cursor = df_business.find()
print(cursor)

In [16]:
def batched(cursor, batch_size):
    batch = []
    for doc in cursor:
        batch.append(doc)
        if batch and not len(batch) % batch_size:
            yield batch
            batch = []

    if batch:   # last documents
        yield batch

df = pd.DataFrame()
for batch in batched(cursor, 10000):
    df = df.append(batch, ignore_index=True)

In [17]:
df_business = df

In [18]:
df_business.isna().sum().sum()

0

In [19]:
df_business.shape

(792133, 12)

In [20]:
df_business.columns

Index(['_id', 'business_id', 'categories', 'city', 'is_open', 'name',
       'review_count', 'stars', 'state', 'business_id_rev', 'review_id',
       'text'],
      dtype='object')

In [21]:
df_business = df_business.drop(['_id'], axis = 1)

In [22]:
df_business['business_id'].nunique()

8731

In [23]:
df_business.head()

,business_id,categories,city,is_open,name,review_count,stars,state,business_id_rev,review_id,text
0,-3yc7i3bvTHv1Zig2rXrSg,"Nightlife, Bars, Steakhouses, Restaurants",St. Petersburg,0,Max & Sam's Bar and Grill,10,1,FL,-3yc7i3bvTHv1Zig2rXrSg,Bhxzfd6LBZ05mI4Z8WiLDg,I want to start off first by saying that this ...
1,-3yc7i3bvTHv1Zig2rXrSg,"Nightlife, Bars, Steakhouses, Restaurants",St. Petersburg,0,Max & Sam's Bar and Grill,10,1,FL,-3yc7i3bvTHv1Zig2rXrSg,qAGeowfbJXokBr62XGWOhQ,Food was good. The busboy was great. The owner...
2,-3yc7i3bvTHv1Zig2rXrSg,"Nightlife, Bars, Steakhouses, Restaurants",St. Petersburg,0,Max & Sam's Bar and Grill,10,1,FL,-3yc7i3bvTHv1Zig2rXrSg,YgDVUpLYjYabBNkgXCq_kg,"So...they lost our reservation, or rather neve..."
3,-3yc7i3bvTHv1Zig2rXrSg,"Nightlife, Bars, Steakhouses, Restaurants",St. Petersburg,0,Max & Sam's Bar and Grill,10,4,FL,-3yc7i3bvTHv1Zig2rXrSg,D69OBe_uS9Soaj0eUtNfyA,You like steak? You will love Max and Sam's. ...
4,-3yc7i3bvTHv1Zig2rXrSg,"Nightlife, Bars, Steakhouses, Restaurants",St. Petersburg,0,Max & Sam's Bar and Grill,10,4,FL,-3yc7i3bvTHv1Zig2rXrSg,2K2GNz70FCLNOWMzRfICMw,"Yummy steaks, friendly service, and two bars! ..."


In [24]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792133 entries, 0 to 792132
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   business_id      792133 non-null  object
 1   categories       792133 non-null  object
 2   city             792133 non-null  object
 3   is_open          792133 non-null  int64 
 4   name             792133 non-null  object
 5   review_count     792133 non-null  int64 
 6   stars            792133 non-null  int64 
 7   state            792133 non-null  object
 8   business_id_rev  792133 non-null  object
 9   review_id        792133 non-null  object
 10  text             792133 non-null  object
dtypes: int64(3), object(8)
memory usage: 66.5+ MB


In [25]:
def clean_text(text):
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)    
    return text

In [26]:
%%time
df_business['text'] = df_business['text'].apply(clean_text)

CPU times: user 2min 3s, sys: 7.55 s, total: 2min 10s
Wall time: 2min 10s


In [29]:
df_business = df_business.loc[df_business.city=='Tampa']

In [30]:
df_business.count()

business_id        303232
categories         303232
city               303232
is_open            303232
name               303232
review_count       303232
stars              303232
state              303232
business_id_rev    303232
review_id          303232
text               303232
dtype: int64

In [31]:
%%time
vectorizer_reviews = CountVectorizer(min_df = .01,max_df = .99, tokenizer = WordPunctTokenizer().tokenize)
vectorized_reviews = vectorizer_reviews.fit_transform(df_business['text'])

CPU times: user 10.3 s, sys: 139 ms, total: 10.5 s
Wall time: 10.5 s


In [32]:
vectorized_reviews.shape

(303232, 865)

In [82]:
vectorized_reviews.todense()

matrix([[ 0,  1,  0, ...,  0,  0,  0],
        [ 1, 12,  0, ...,  1,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0, 10,  0, ...,  0,  0,  0],
        [ 5, 12,  1, ...,  1,  0,  0]])

In [33]:
' | '.join(vectorizer_reviews.get_feature_names_out()[:100]) # only the first 100

'! | + | - | 00 | 1 | 10 | 12 | 15 | 2 | 20 | 3 | 30 | 4 | 5 | 50 | 6 | 7 | 8 | : | ; | a | able | about | absolutely | accommodating | across | actually | add | added | addition | afternoon | again | ago | ahead | all | almost | along | already | also | although | always | am | amazing | ambiance | amount | and | another | anyone | anything | anywhere | appetizer | appetizers | are | area | around | arrived | as | ask | asked | ate | atmosphere | attention | attentive | authentic | available | average | avocado | away | awesome | awful | back | bacon | bad | baked | bar | bartender | bartenders | based | basically | bay | bbq | be | beans | beautiful | beef | beer | beers | before | behind | believe | best | better | beyond | big | bill | birthday | bit | bite | bites | black'

In [34]:
df_business_only = db['recommender_business']
print(df_business_only)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Yelp_Database'), 'recommender_business')


In [35]:
cursor = df_business_only.find()
print(cursor)

In [36]:
def batched(cursor, batch_size):
    batch = []
    for doc in cursor:
        batch.append(doc)
        if batch and not len(batch) % batch_size:
            yield batch
            batch = []

    if batch:   # last documents
        yield batch

df = pd.DataFrame()
for batch in batched(cursor, 10000):
    df = df.append(batch, ignore_index=True)

In [37]:
df_business_only=df

In [38]:
df_business_only

,_id,business_id,categories,city,is_open,name,review_count,stars,state
0,640ba2188e928c352265d533,eEOYSgkmpB90uNA7lDOMRA,"Vietnamese, Food, Restaurants, Food Trucks",Tampa Bay,1,Vietnamese Food Truck,10,4.0,FL
1,640ba2188e928c352265d534,0bPLkL0QhhPO5kt1_EXmNQ,"Food, Delis, Italian, Bakeries, Restaurants",Largo,0,Zio's Italian Market,100,4.5,FL
2,640ba2188e928c352265d535,uI9XODGY_2_ieTE6xJ0myw,"Restaurants, American (New), Italian",Tampa,0,Roman Forum,23,4.0,FL
3,640ba2188e928c352265d536,JgpnXv_0XhV3SfbfB50nxw,"Restaurants, Pizza",Tampa,0,Joe's Pizza,35,4.0,FL
4,640ba2188e928c352265d537,pJfh3Ct8iL58NZa8ta-a5w,"Burgers, Sports Bars, Bars, Lounges, Restauran...",Wesley Chapel,1,Top Shelf Sports Lounge,95,4.5,FL
...,...,...,...,...,...,...,...,...,...
8726,640ba2188e928c352265f749,Scd-rcsQCn60t1sHHFv-og,"Cafes, Restaurants, Breakfast & Brunch, Americ...",St. Petersburg,1,First Watch,183,3.5,FL
8727,640ba2188e928c352265f74a,8MzF1Tlgz0pOkxmhP5dYzA,"American (Traditional), Burgers, Restaurants",St. Petersburg,1,El Cap Restaurant,414,3.5,FL
8728,640ba2188e928c352265f74b,-bZQH8yjm7ntTyGeLQwh8Q,"Sandwiches, Restaurants, Diners",Largo,0,Farmer's Kitchen Restaurant,6,4.0,FL
8729,640ba2188e928c352265f74c,BIyT7Kr7tMJqlfp4oOOYQg,"Breakfast & Brunch, Cafes, Restaurants",Riverview,0,Copper Bell Cafe,49,3.5,FL


In [41]:
df_business_only = df_business_only.loc[df_business_only.city=='Tampa']

In [89]:
%%time
vectorizer_categories = CountVectorizer(min_df = 1, max_df = 1., tokenizer = lambda x: x.split(', '))
vectorized_categories = vectorizer_categories.fit_transform(df_business_only['categories'])

CPU times: user 3.64 ms, sys: 7.2 ms, total: 10.8 ms
Wall time: 10.1 ms


In [92]:
vectorized_categories.nonzero()

(array([   0,    0,    0, ..., 2959, 2959, 2959], dtype=int32),
 array([287,   8, 194, ..., 321,  19, 335], dtype=int32))

In [91]:
vectorized_categories.shape

(2960, 372)

In [93]:
' | '.join(vectorizer_categories.get_feature_names_out()[:100]) # only the first 100

"acai bowls | active life | adult | adult entertainment | afghan | african | airport terminals | airports | american (new) | american (traditional) | amusement parks | appliances | arabic | arcades | argentine | armenian | art galleries | arts & crafts | arts & entertainment | asian fusion | australian | austrian | auto loan providers | auto repair | automotive | bagels | bakeries | bangladeshi | banks & credit unions | bar crawl | barbeque | bars | bartenders | basque | beaches | beauty & spas | bed & breakfast | beer | beer bar | beer gardens | beer tours | belgian | beverage store | bike rentals | boat charters | body shops | books | bookstores | bowling | brasseries | brazilian | breakfast & brunch | breweries | brewpubs | british | bubble tea | buddhist temples | buffets | building supplies | burgers | burmese | bus tours | business law | butcher | cafes | cafeteria | cajun/creole | candy stores | cantonese | car dealers | caribbean | casinos | caterers | champagne bars | cheese s

In [94]:
df_business['business_id'].values

array(['0-M3vGd6IveJh77-Bwc8EQ', '0-M3vGd6IveJh77-Bwc8EQ',
       '0-M3vGd6IveJh77-Bwc8EQ', ..., 'z_7TZeTXsDH6MmkPHQN8nQ',
       'z_7TZeTXsDH6MmkPHQN8nQ', 'z_7TZeTXsDH6MmkPHQN8nQ'], dtype=object)

In [95]:
%%time
from scipy import sparse
businessxreview = sparse.csr_matrix(pd.get_dummies(df_business['business_id'].values))

CPU times: user 7.91 s, sys: 741 ms, total: 8.65 s
Wall time: 8.67 s


In [96]:
print('restuarants x categories: \t', vectorized_categories.shape) 
print('restuarants x reviews: \t\t' , businessxreview.shape) 
print('reviews x words: \t\t', vectorized_reviews.shape)

restuarants x categories: 	 (2960, 372)
restuarants x reviews: 		 (303232, 2960)
reviews x words: 		 (303232, 865)


Now we are ready to choose a seed restaurant and find  other restaurants that might be as good as the seed restaurant. We make sure to choose a restaurant with good number of reviews and ratings.

In [97]:
# df_business_review = df_business_review.drop(["_id"], axis = 1)
df_business.columns

Index(['business_id', 'categories', 'city', 'is_open', 'name', 'review_count',
       'stars', 'state', 'business_id_rev', 'review_id', 'text'],
      dtype='object')

In [98]:
# to choose a restaurant, just copy the business id and paste it in the next cell
# you can always rerun the cell to choose another restuarant. 
df_business.sample(7)

,business_id,categories,city,is_open,name,review_count,stars,state,business_id_rev,review_id,text
361504,VSpseb6oijtKffRRL_xfgQ,"Soup, Restaurants, Cuban, Sandwiches",Tampa,1,The Stone Soup Company,529,4,FL,VSpseb6oijtKffRRL_xfgQ,yxo19YbdtpJtmJrSG3Fr0A,great service cool setting tons menu choices c...
649714,oRZzKWdyMQld6N_y4UWkAg,"Latin American, Restaurants, Peruvian, Seafood",Tampa,1,Terra Sur Cafe,316,4,FL,oRZzKWdyMQld6N_y4UWkAg,oEb2PcSTxYOBWf3MqYuHTA,finally experienced terra sur questionable eat...
392943,1N9WC6FwF8MSc7BHbIP35A,"Desserts, Restaurants, Vegan, Food, Sandwiches...",Tampa,1,Vegan International Co. Kitchen & Market,269,5,FL,1N9WC6FwF8MSc7BHbIP35A,nuYpDMbjGX7emem_L0kLZQ,wow wow wow ! usually restaurants either great...
642900,SwBhaxfQPbyhsi0QHUAN0A,"Food, Hotels, Coffee & Tea, Hotels & Travel, R...",Tampa,1,Seminole Hard Rock Hotel & Casino Tampa,739,1,FL,SwBhaxfQPbyhsi0QHUAN0A,WS9bcQXQiNQUvf5_NQSdlw,1 10 rude staff hotel reservations players clu...
629783,03dIEDBsDERr6Osmey-udQ,"Seafood, Steakhouses, Southern, American (New)...",Tampa,1,Steelbach at Armature Works,727,2,FL,03dIEDBsDERr6Osmey-udQ,vrFbhoZxmlVLf4Ye7vAIrw,restaurant good think is first time went food ...
399479,7etwdmBaquL2l6NfGaqw0Q,"Korean, Sushi Bars, Restaurants",Tampa,0,Han Il Kwan Korean Restaurant,10,5,FL,7etwdmBaquL2l6NfGaqw0Q,WIHZdT9DRIvf-7yPaM9IeQ,eat twice month food excellent especially like...
600100,BEMSDLPP630Fpdw_QpqLVA,"Creperies, Desserts, Breakfast & Brunch, Resta...",Tampa,1,La Creperia Cafe,350,5,FL,BEMSDLPP630Fpdw_QpqLVA,P3oqW2E2L7YTdXSSVaOTgw,get pollo verde panini great !


In [100]:
business_choose = 'VSpseb6oijtKffRRL_xfgQ' # business_id of desired business

In [101]:
new_reviews = df_business.loc[df_business['business_id'] == business_choose, 'text']

In [102]:
print('\n'.join([r[:100] for r in new_reviews.tolist()])) # restaurant reviews

really excited try out menu promising however food average got grilled cheese panini potato salad ev
found place googling soup boyfriend drove minutes get glad did got special - - italian wedding soup 
passed stone soup many times walking ybor never tried decided check out waiter awesome friendly atte
appropriate various dietary persuasions i have eaten several times never disappointed extra cool poi
best grilled cheese town my + boyfriend alway gets real cuban award winner area loves
chose try restaurant visited ybor city good yelp reviews inside looked nice ample sitting areas bart
found place walking street ybor city like before ! ! ! + i love place ! jessica bartender took care 
tampa conference looking good cuban sandwich stone soup company popped yelp i am fan book figured wo
great atmosphere huge beer selection awesome imports craft beers great sandwiches salads too side di
solid stars fish po boy tasty generous size bonus points subversive menu 
food mediocre chicken soup pract

In [103]:
new_categories = df_business.loc[df_business['business_id'] == business_choose, 'categories']

In [104]:
set(new_categories.tolist()) #  restaurant categories

{'Soup, Restaurants, Cuban, Sandwiches'}

Here, we compute two sets of distancecs: we compute the correlation distance of the average vectorized reviews to all the reviews, and compute the correlation distance between this category and all other categories. The category trick will be clearer when we see the results.

In [105]:
businessxreview.shape

(303232, 2960)

In [106]:
businessxreview.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [107]:
from scipy.spatial.distance import cdist
# find most similar reviews
dists1 = cdist(vectorizer_reviews.transform(new_reviews).todense().mean(axis=0), 
              vectorized_reviews.T.dot(businessxreview).T.todense(), 
               metric='correlation')
# find most similar categories
dists2 = cdist(vectorizer_categories.transform(new_categories).todense().mean(axis=0), 
              vectorized_categories.todense(), 
               metric='correlation')

In [108]:
dists1.shape

(1, 2960)

In [109]:
dists2.shape

(1, 2960)

In [110]:
dists2

array([[0.71798333, 0.65123603, 0.75815217, ..., 0.75815217, 0.75815217,
        0.82261819]])

Now we combine the two sets of distances and take the average of those (we can take other metrics such as min or max, depending on your priority).

In [111]:
# combine the two vectors in one matrix
dists_together = np.vstack([dists1.ravel(), dists2.ravel()]).T

In [112]:
dists_together

array([[0.24040151, 0.71798333],
       [0.21335687, 0.65123603],
       [0.58795558, 0.75815217],
       ...,
       [0.21650656, 0.75815217],
       [0.3820175 , 0.75815217],
       [0.24022049, 0.82261819]])

In [113]:
# this is a key cell: how are we going to prioritize ?
dists = dists_together.mean(axis=1)

In [114]:
dists

array([0.47919242, 0.43229645, 0.67305388, ..., 0.48732937, 0.57008484,
       0.53141934])

selecting 10 restaurants most similar to the selected restaurant.

In [115]:
# select the closest 10
closest = dists.argsort().ravel()[:10]

Selected restaurants display:

In [118]:
df_business_only.loc[df_business_only['business_id']== business_choose, ['business_id', 'categories', 'name', 'stars']]

,business_id,categories,name,stars
6048,VSpseb6oijtKffRRL_xfgQ,"Soup, Restaurants, Cuban, Sandwiches",The Stone Soup Company,4.0


**Best Matches to the vector of the chosen restaurants :**

In [119]:
df_business_only.loc[df_business_only['business_id'].isin(df_business_only['business_id'].iloc[closest]), ['business_id', 'categories', 'name', 'stars']]

,business_id,categories,name,stars
2065,Tn_OK04DzbDQij9ZpMlT9Q,"Restaurants, Cuban, Sandwiches",The Floridian,3.5
5031,dk2QvuF2blpv0j7ljFwcaQ,"Sandwiches, Restaurants",Sub Port Giant New Jersey Submarine Sandwiches,4.5
6048,VSpseb6oijtKffRRL_xfgQ,"Soup, Restaurants, Cuban, Sandwiches",The Stone Soup Company,4.0
6506,-SCIBOTjKGJXmeMMPi-uoQ,"Spanish, Cuban, Sandwiches, Restaurants",Papi's Cafe,5.0
6566,pr1-qcxYNoU2iNcRPr3Bfg,"Restaurants, Cuban, Sandwiches",Havana Village Sandwich Shop,3.0
7026,OvEG3xcmO0Vc2WoAvCPUAA,"Restaurants, Cuban, Sandwiches",Stadium Cafe,4.0
7896,wnAEZpQ38XgHElOz4Vd6UQ,"Sandwiches, Soup, Restaurants",Gonzo's Sandwich Shop,4.5
8310,lgop6Pbyvk1qi8CLclQEeg,"Restaurants, Sandwiches",Sun's Cafe,4.5
8465,pP3QrwJowWoYsVc6zfsW0g,"Cuban, Restaurants",Versalles,2.0
8674,OAXfyHD2TUE0CRnvLPhjrQ,"Restaurants, Cuban",Havana 86,4.0


## Exporting models and sparse matrices for deployment:

In [121]:
import scipy

In [122]:
scipy.sparse.save_npz('businessxreview.npz', businessxreview, compressed=True)

In [123]:
scipy.sparse.save_npz('vectorized_reviews.npz', vectorized_reviews, compressed=True)

In [125]:
scipy.sparse.save_npz('vectorized_categories.npz', vectorized_categories, compressed=True)

In [126]:
df_business.to_csv('df_business.csv',index=False)

In [127]:
df_business_only.to_csv('df_business_only.csv',index=False)

In [129]:
import joblib

# example for saving python object as pkl
joblib.dump(vectorizer_reviews, "vectorizer_reviews.pkl")

['vectorizer_reviews.pkl']